## CSC 580: Critical Thinking 4 - Toxicology Testing
TODO - Description

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from deepchem import deepchem as dc
from sklearn.metrics import accuracy_score

# Seeds are set via assignment parameters
np.random.seed(456)
tf.random.set_seed(456)


#### Preparing the Data
Using the [Tox21 Dataset](https://tox21.gov/resources/)

In [4]:
_,(train, valid, test),_ = dc.molnet.load_tox21()

train_X, train_y, train_w = train.X, train.y, train.w
valid_x, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w

RDKit WARNING: [13:18:15] WARNING: not removing hydrogen atom without neighbors
[13:18:15] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [13:19:13] WARNING: not removing hydrogen atom without neighbors
[13:19:13] WARNING: not removing hydrogen atom without neighbors


In [5]:
train_y = train_y[:,0]
valid_y = valid_y[:,0]
test_y = test_y[:,0]

train_w = train_w[:,0]
valid_w = valid_w[:,0]
test_w = test_w[:,0]

#### Defining Tensorflow Graph

In [ ]:
d = 1024
n_hidden = 50
learning_rate = .001
n_epochs = 10
batch_size = 100

with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, d))
    y = tf.placeholder(tf.float32, (None,))

with tf.name_scope("hidden-layer"):
    W = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden = tf.nn.relu(tf.matmul(x,W) + b)

with tf.name_scope("output"):
    W = tf.Variable(tf.random_normal((n_hidden, 1)))
    b = tf.Variable(tf.random_normal((1,)))
    y_logit = tf.matmul(x_hidden,W) + b

    # The sigmoid gives the class probability of 1
    y_one_prob = tf.sigmoid(y_logit)

    # Rounding P(y=1) will give the correct prediction
    y_pred = tf.round(y_one_prob)

with tf.name_scope("loss"):
    # Compute the cross-entropy term for each datapoint
    y_expand = tf.expand_dims(y, 1)
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)

    # Sum all contributions
    l = tf.reduce_sum(entropy)

with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()